In [25]:
from dotenv import load_dotenv
import os
import json 
import requests

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from pymongo.mongo_client import MongoClient
from typing import Optional

from google.cloud import storage
from google.oauth2 import service_account
import urllib.parse

from shapely.geometry import MultiPolygon, Polygon
from shapely import wkt
from PIL import Image, ImageDraw

In [2]:
load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
uri = os.getenv('MONGO')
#AZURE_EMBEDDING = os.getenv("AZURE_EMBEDDING")

In [3]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    openai_api_version="2024-03-01-preview",
    temperature=0.1,
)

In [26]:
def get_report():
    with open("emergency_details.json", "r") as file:
        emergency_details = json.load(file)
    
    details = "\n".join([f"- '{key}': {value}" for key, value in emergency_details.items()])
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You will read in the emergency_details.json.
                Details:
                {details}
                There are 8 fields in the JSON file.
                - 'name': The name of the person.
                - 'appearance': A description of the person's appearance.
                - 'address': The address where assistance is needed.
                - 'situation': A detailed description of the situation facing the person, written in third person.
                - 'latitude': The latitude of the address mentioned above.
                - 'longitude': The longitude of the address mentioned above.
                - 'classify': The level of severity of the address. There are four levels: destroyed, major-damage, minor-damage and no-damage.
                - 'destroyed': The percentage of destruction in the surrounding area. The higher the percentage, the more severe the damage.
                According to the information above, please write a message in detail to the emergency services to inform them of the situation.
                """
            ),
            ("human", ""),
        ]
    )
    chain = (
        {
            "details": RunnablePassthrough()
        }
        | prompt
        | llm
    )
    response = chain.invoke(details)
    return response

In [27]:
get_report()

AIMessage(content="Subject: Urgent Assistance Required at 2402 Wisconsin Ave, Joplin, MO\n\nDear Emergency Services,\n\nI am writing to report a critical situation that requires immediate attention at the following location:\n\nAddress: 2402 Wisconsin Ave, Joplin, MO 64804\nLatitude: 37.0640558\nLongitude: -94.50263369999999\n\nThe area has been classified as 'destroyed' with a reported 62.80% of the surrounding structures and infrastructure severely damaged due to the joplin-tornado incident. This level of destruction indicates that the location is likely to be in a state of considerable disarray, with potential risks to any remaining occupants and a pressing need for search and rescue operations.\n\nAs of now, we do not have specific details about the individuals involved or the exact nature of their situation, as the 'name' and 'situation' fields in the report are empty. There is also no description of the person's appearance under 'appearance'. Despite the lack of these details, th

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Listen carefully to the user's explanation of their emergency situation. Extract the following details and format them into simple string responses:
            - 'name': The name of the person.
            - 'appearance': A description of the person's appearance.
            - 'address': The address where assistance is needed.
            - 'situation': A detailed description of the situation facing the person, written in third person.
            Format the information neatly as a JSON structure with each detail clearly noted as individual strings. If any detail is missing, use 'None' for that field. Do not make assumptions to fill in missing information.
            """
        ),
        ("human", "{question}"),
    ]
)

class EmergencyInfo(BaseModel):
    """Information Extracted from Emergency Situation"""
    name: Optional[str] = Field(None, description="Name of the person involved in the emergency")
    appearance: Optional[str] = Field(None, description="Description of the person's appearance")
    address: Optional[str] = Field(None, description="Address where the emergency is occurring")
    situation: Optional[str] = Field(None, description="Detailed description of the emergency situation, described in third person")


parser = JsonOutputParser(pydantic_object=EmergencyInfo)

chain = prompt | llm | parser

In [7]:
def geocode_address(api_key, address):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': address,
        'key': api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        return response.status_code, response.text

    info = response.json()
    if info['status'] == 'OK' and info['results']:
        location = info['results'][0]['geometry']['location']
        coordinates = {'lat': location['lat'], 'lng': location['lng']}
        
        # with open('coordinates.json', 'w') as f:
        #     json.dump(coordinates, f, indent=4)  
        return coordinates
    else:
        return "No location found or error in response."

In [8]:
def handle_emergency_situation(question):
    response = chain.invoke(question)
    #print(response)
    address = response['address']
    geocode_result = geocode_address(api_key, address)
    
    if isinstance(geocode_result, tuple):
        print(f"Error {geocode_result[0]}: {geocode_result[1]}")
    elif isinstance(geocode_result, str):
        print(geocode_result)
    else:
        # Successful geocode
        return geocode_result


### Dead

In [43]:
# def find_image_urls_containing_point(images_collection, lat, lon):
#     point = {
#         "type": "Point",
#         "coordinates": [lon, lat]  # Ensure longitude is first
#     }

#     # Using the `images_collection` that was passed to the function
#     image = images_collection.find_one({
#         "bounding_box": {
#             "$geoIntersects": {
#                 "$geometry": point
#             }
#         }
#     })

#     if image:
#         print("Image found:", image['name'])
#         return image['pre'], image['post']
#     else:
#         print("No image found containing the point.")
#         return None, None


In [46]:
# def find_image_urls_containing_point(images_collection, lat, lon):
#     point = {
#         "type": "Point",
#         "coordinates": [lon, lat]  # Ensure longitude is first
#     }
#     image = images_collection.find_one({
#         "bounding_box": {
#             "$geoIntersects": {
#                 "$geometry": point
#             }
#         }
#     })
#     if image:
#         print("Image found:", image['name'])
#         bounding_box_coords = image['bounding_box']['coordinates'][0]
#         northwest = bounding_box_coords[0]  # Min longitude, Max latitude
#         northeast = bounding_box_coords[1]  # Max longitude, Max latitude
#         southeast = bounding_box_coords[2]  # Max longitude, Min latitude
#         southwest = bounding_box_coords[3]  # Min longitude, Min latitude

#         return {
#             'pre_url': image['pre'],
#             'post_url': image['post'],
#             'northwest': northwest,
#             'northeast': northeast,
#             'southeast': southeast,
#             'southwest': southwest
#         }
#     else:
#         print("No image found containing the point.")
#         return None

# start

In [9]:
def retrieve_image(address):
    # Extract coordinates from the address
    coords = handle_emergency_situation(address)
    latitude = coords['lat']
    longitude = coords['lng']
    print(f"Latitude: {latitude}, Longitude: {longitude}")
    client = MongoClient(uri)
    db = client['ncsa']
    images_collection = db['ncsa'] 

    point = {
        "type": "Point",
        "coordinates": [longitude, latitude]  # Ensure longitude is first
    }
    image = images_collection.find_one({
        "bounding_box": {
            "$geoIntersects": {
                "$geometry": point
            }
        }
    })
    if image:
        print("Image found:", image['name'])
        bounding_box_coords = image['bounding_box']['coordinates'][0]
        northwest = bounding_box_coords[0]  # Min longitude, Max latitude
        northeast = bounding_box_coords[1]  # Max longitude, Max latitude
        southeast = bounding_box_coords[2]  # Max longitude, Min latitude
        southwest = bounding_box_coords[3]  # Min longitude, Min latitude

        return {
            'pre_url': image['pre'],
            'post_url': image['post'],
            'northwest': northwest,
            'northeast': northeast,
            'southeast': southeast,
            'southwest': southwest
        }
    else:
        print("No image found containing the point.")
        return None


In [10]:
def get_credentials():
    json_file_path = 'ncsa-hackathon-f88e88a84517.json'
    with open(json_file_path, 'r') as file:
        credentials_dict = json.load(file)
    return service_account.Credentials.from_service_account_info(credentials_dict)

def download_blob(bucket_name, source_blob_name, destination_file_name):
    credentials = get_credentials()
    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

In [11]:
def get_images(address, download_path):
    # Call retrieve_image to get the image URLs and other details
    image_data = retrieve_image(address)
    if image_data is None:
        print("No image found for the specified address.")
        return

    bucket_name = 'ncsa-hackthon'
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    def get_blob_name_from_url(url):
        parsed_url = urllib.parse.urlparse(url)
        path_parts = parsed_url.path.split('/')
        blob_name = '/'.join(path_parts[2:])  # Adjust if bucket structure is different
        return blob_name

    # Retrieve the blob names from URLs
    pre_blob_name = get_blob_name_from_url(image_data['pre_url'])
    post_blob_name = get_blob_name_from_url(image_data['post_url'])

    # Get filenames from the blob names
    pre_file_name = pre_blob_name.split('/')[-1]
    post_file_name = post_blob_name.split('/')[-1]

    # Construct local file paths for download
    pre_path = os.path.join(download_path, pre_file_name)
    post_path = os.path.join(download_path, post_file_name)

    # Download the images to the specified path
    download_blob(bucket_name, pre_blob_name, pre_path)
    download_blob(bucket_name, post_blob_name, post_path)
    
    return image_data['northwest'], image_data['northeast'], image_data['southeast'], image_data['southwest']

In [12]:
def latlng_to_pixels(lat, lng, img_bounds, img_size):
    min_lat, max_lat, min_lng, max_lng = img_bounds
    
    lat_scale = img_size[1] / (max_lat - min_lat)
    lng_scale = img_size[0] / (max_lng - min_lng)
    
    x = (lng - min_lng) * lng_scale
    y = (max_lat - lat) * lat_scale  
    return (x, y)

def get_bounds_and_draw(img_bounds, lat, lng, img_path):
    # Calculate the image size
    img = Image.open(img_path)
    img_size = img.size
    pixel_coords = latlng_to_pixels(lat, lng, img_bounds, img_size)

    # Drawing on the image
    draw = ImageDraw.Draw(img)
    radius = 20
    color = 'red'
    draw.ellipse((pixel_coords[0] - radius, pixel_coords[1] - radius, pixel_coords[0] + radius, pixel_coords[1] + radius), outline=color, width=3)

    # Save the modified image
    modified_img_path = 'marked/img.png'
    img.save(modified_img_path)

In [13]:
text = "I'm stuck under a building at 2400 Wisconsin Ave, right next to Joplin High School"
download_path = "data"
nw, ne, se, sw = get_images(text, download_path)

img_bounds = (sw[1], ne[1], nw[0], se[0])
lat = 37.0640558  # Example latitude
lng = -94.5026337  # Example longitude
img_path = 'pred34_cls_/joplin-tornado_00000001_pre_disaster_part2.png.png'

get_bounds_and_draw(img_bounds, lat, lng, img_path)

Latitude: 37.064216, Longitude: -94.50229619999999
Image found: joplin-tornado_00000001_pre_disaster
Blob image/joplin-tornado_00000001_pre_disaster.png downloaded to data/joplin-tornado_00000001_pre_disaster.png.
Blob image/joplin-tornado_00000001_post_disaster.png downloaded to data/joplin-tornado_00000001_post_disaster.png.
